In [ ]:

import samna
from collections import Counter
from torch.utils.data import Subset
import sinabs.layers as sl
from torch import nn
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential
from tonic.datasets.nmnist import NMNIST

from tonic.transforms import ToFrame
from tonic.datasets import nmnist

import sinabs.layers as sl
from torch import nn
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential
import samna
import samnagui
import time
import torch
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

from torch import nn
from multiprocessing import Process
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork


# download dataset
root_dir = "./dataset/"
_ = NMNIST(save_to=root_dir, train=True)

In [5]:

batch_size = 4

# just replace the ReLU layer with the sl.IAFSqueeze
snn_bptt = nn.Sequential(
    # [2, 34, 34] -> [8, 17, 17]
    nn.Conv2d(in_channels=2, out_channels=8, kernel_size=(3, 3), padding=(1, 1), bias=False),
    sl.IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, surrogate_grad_fn=PeriodicExponential()),
    nn.AvgPool2d(2, 2),
    # [8, 17, 17] -> [16, 8, 8]
    nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=(1, 1), bias=False),
    sl.IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, surrogate_grad_fn=PeriodicExponential()),
    nn.AvgPool2d(2, 2),
    # [16 * 8 * 8] -> [16, 4, 4]
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=(1, 1), stride=(2, 2),  bias=False),
    sl.IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, surrogate_grad_fn=PeriodicExponential()),
    # [16 * 4 * 4] -> [10]
    nn.Flatten(),
    nn.Linear(16 * 4 * 4, 10, bias=False),
    sl.IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, surrogate_grad_fn=PeriodicExponential()),
)

# init the model weights
for layer in snn_bptt.modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_normal_(layer.weight.data)

        n_time_steps = 100
to_raster = ToFrame(sensor_size=NMNIST.sensor_size, n_time_bins=n_time_steps)

snn_train_dataset = NMNIST(save_to=root_dir, train=True, transform=to_raster)
snn_test_dataset = NMNIST(save_to=root_dir, train=False, transform=to_raster)


In [7]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

snn_bptt = torch.load("./model/snn_bptt_fullmodel.pth",map_location="cpu")
snn_bptt.eval().to(device)


Sequential(
  (0): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=4, num_timesteps=-1)
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=4, num_timesteps=-1)
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (7): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=4, num_timesteps=-1)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=256, out_features=10, bias=False)
  (10): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Pa

In [17]:
from sinabs.backend.dynapcnn.dynapcnn_visualizer import DynapcnnVisualizer

import samna
# first define a callback function to modify the devkit configuration
# the callback function should only has 1 devkit config instance as its input argument
def config_modify_callback(devkit_cfg):

    # enable visualizing the output from dvs(pre-processing) layer
    devkit_cfg.dvs_layer.monitor_enable = True
    # disable visualizing the events generated by the embedded dvs on Speck
    devkit_cfg.dvs_layer.raw_monitor_enable = False
    # prevent the events generated by the embedded dvs been feed to the DynapCNN Core.
    devkit_cfg.dvs_layer.pass_sensor_events = False
    # point the dvs layer output destination to the core#0 
    devkit_cfg.dvs_layer.destinations[0].enable = True
    devkit_cfg.dvs_layer.destinations[0].layer = 0

    # the callback must return the modified devkit config
    return devkit_cfg

# close the devkit before reopen
# samna.device.close_device(dynapcnn.samna_device)
cpu_snn = snn_bptt.to(device="cpu")
devkit_name = "speck2fdevkit"
# init DynapcnnNetwork instance
dynapcnn = DynapcnnNetwork(snn=cpu_snn, input_shape=(2, 34, 34), discretize=True, dvs_input=True)

# define which layers output you want to monitor
layers_to_monitor = [0, 1, 2, 3]
# pass the callback function into the `.to` method
dynapcnn.to(device=devkit_name, chip_layers_ordering=[0, 1, 2, 3], monitor_layers=layers_to_monitor, config_modifier=config_modify_callback)
print(f"The SNN is deployed on the core: {dynapcnn.chip_layers_ordering}")

visualizer = DynapcnnVisualizer(
    window_scale=(4, 8),
    dvs_shape=(34, 34),
    spike_collection_interval=50,
)


Network is valid
The SNN is deployed on the core: [0, 1, 2, 3]


In [19]:
# Restart the devkit
import samna
import samnagui
import time
import torch
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

from torch import nn
from multiprocessing import Process
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork

# get device name
devices = samna.device.get_all_devices()
device_names = [each.device_type_name for each in devices]
device_names

# open devkit
devkit = samna.device.open_device(device_names[0])

# get the handle of the stop-watch of the devkit
stop_watch = devkit.get_stop_watch()

# get the handle of the power monitor of the devkit
power_monitor = devkit.get_power_monitor()

# create samna node for power reading
power_source_node = power_monitor.get_source_node()
power_buffer_node = samna.BasicSinkNode_unifirm_modules_events_measurement()

# init graph
samna_graph = samna.graph.EventFilterGraph()

import time

# build graph
samna_graph.sequential([power_source_node, power_buffer_node])
time.sleep(0.5)


In [20]:
import time
from collections import Counter
from torch.utils.data import Subset
from tqdm import tqdm

# ====== Utility function ======
def measure_average_power(duration=5.0, sample_rate=100.0):
    power_monitor.start_auto_power_measurement(sample_rate)
    stop_watch.set_enable_value(True)
    power_buffer_node.get_events()
    time.sleep(duration)
    power_monitor.stop_auto_power_measurement()
    power_events = power_buffer_node.get_events()

    num_tracks = 5
    power_sum = [0.0] * num_tracks
    power_count = [0] * num_tracks
    for evt in power_events:
        power_sum[evt.channel] += evt.value
        power_count[evt.channel] += 1
    
    avg_uw = [power_sum[i] / power_count[i] * 1e6 if power_count[i] > 0 else 0.0 for i in range(num_tracks)]
    total_mw = sum(avg_uw) / 1000.0  # Convert μW to mW
    return avg_uw, total_mw

# ====== Step 1: Idle Power ======
print("Measuring IDLE power...")

try:
    power_monitor.stop_auto_power_measurement()
    samna_graph.stop()
except:
    pass

samna_graph.start()
stop_watch.set_enable_value(True)
power_buffer_node.get_events()

idle_avg_uw, idle_total_mw = measure_average_power(duration=5.0, sample_rate=100.0)
samna_graph.stop()

print(f"[IDLE] Total: {idle_total_mw:.2f} mW")
print(f"[IDLE] Per Track (μW): {idle_avg_uw}")

# ====== Step 2: Active Power + Inference ======
print("\nRunning inference and measuring ACTIVE power...")

samna_graph.start()
stop_watch.set_enable_value(True)
power_buffer_node.get_events()
power_monitor.start_auto_power_measurement(1.0)

start_time = time.time()

snn_test_dataset = NMNIST(save_to=root_dir, train=False)
subset_indices = list(range(0, len(snn_test_dataset), 1000))
snn_test_dataset = Subset(snn_test_dataset, subset_indices)

correct_samples = 0
test_samples = 0

for events, label in tqdm(snn_test_dataset):
    spike_events = []
    for ev in events:
        spk = samna.speck2f.event.Spike()
        spk.x, spk.y, spk.timestamp, spk.feature = ev["x"], ev["y"], ev["t"] - events["t"][0], ev["p"]
        spk.layer = 0
        spike_events.append(spk)

    output_events = dynapcnn(spike_events)
    layer3_spks = [e.feature for e in output_events if e.layer == 3]
    prediction = Counter(layer3_spks).most_common(1)[0][0] if layer3_spks else -1
    if prediction == label:
        correct_samples += 1
    test_samples += 1

end_time = time.time()

power_monitor.stop_auto_power_measurement()
samna_graph.stop()

active_power_events = power_buffer_node.get_events()

# ====== Step 3: Calculate Active Power ======
num_tracks = 5
power_sum = [0.0] * num_tracks
power_count = [0] * num_tracks
for evt in active_power_events:
    power_sum[evt.channel] += evt.value
    power_count[evt.channel] += 1

active_avg_uw = [power_sum[i] / power_count[i] * 1e6 if power_count[i] > 0 else 0.0 for i in range(num_tracks)]
active_total_mw = sum(active_avg_uw) / 1000.0

# ====== Step 4: Final Metrics ======
dynamic_mw = active_total_mw - idle_total_mw
elapsed_time = end_time - start_time
fps = test_samples / elapsed_time
fps_per_watt = fps / (dynamic_mw / 1000.0) if dynamic_mw > 0 else float('inf')

# ====== Step 5: Print Summary ======
print("\n===== Summary =====")
print(f"Idle Power:    {idle_total_mw:.2f} mW")
print(f"Active Power:  {active_total_mw:.2f} mW")
print(f"Dynamic Power: {dynamic_mw:.2f} mW")
print(f"On-chip Accuracy: {correct_samples / test_samples:.2%}")
print(f"FPS: {fps:.2f}")
print(f"FPS per Watt: {fps_per_watt:.2f}")
print("===================")


`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.


Measuring IDLE power...


`set_enable_value` is DEPRECATED, use `start` and `stop` instead.


[IDLE] Total: 1.20 mW
[IDLE] Per Track (μW): [40.429017825690465, 70.41776074470337, 279.214609464464, 65.38956819889775, 749.2698846353404]

Running inference and measuring ACTIVE power...


 10%|█         | 1/10 [00:01<00:11,  1.26s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
 20%|██        | 2/10 [00:02<00:09,  1.21s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
 30%|███       | 3/10 [00:03<00:08,  1.19s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
 40%|████      | 4/10 [00:04<00:07,  1.21s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
 50%|█████     | 5/10 [00:06<00:06,  1.21s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
`set_enable_value` is DEPRECATED, use `start` and `stop` instead.
 60%|██████    | 6/10 [00:07<00:04,  1.20s/it]`set_enable_value` is DEPRECATED, use `start` and `stop` instead


===== Summary =====
Idle Power:    1.20 mW
Active Power:  1.25 mW
Dynamic Power: 0.05 mW
On-chip Accuracy: 0.00%
FPS: 0.82
FPS per Watt: 16662.16
